In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm
from sklearn.preprocessing import MinMaxScaler
import pickle

import math
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from pmdarima.arima.utils import ndiffs,nsdiffs


In [ ]:
# Read the dataset from the excel file
file=pd.read_excel('sample.xlsx',sheet_name='Data',index_col=0)
def data_cleaning(data):
    """
        Method Name: data_cleaning
        Description: This function carried out data_cleaing by removing special characters 
        from the dataset and arranging the data in an orderly manner.
        Output: df_store= DataFrame containing initial data
    """
    data=data.T #Transpose the data
    to_remove=['[',']',','] # Characters to remove from the strings of the feature
    for v in to_remove:
        for l in range (data.shape[0]):
            for m in range(data.shape[1]):
                value=data.iloc[l,m].replace(v,'')
                data.iloc[l,m]=value
    df_store=[]
    for i in range(len(data.columns)):
        # Split the data in every feature by a single space
        a=data.iloc[0,i].split(' ')
        b=data.iloc[1,i].split(' ')
        df=pd.DataFrame([a,b]).T
        df.columns=['ds','y'] # Assign the column name of the dataframe
        df=df.astype('float64') # Change the dataset type to float
        df['ds']=pd.to_datetime(df['ds'], unit='s') # Make the feature datatype as datetime
        df.to_csv('sample{x}_time.csv'.format(x=i)) # Save the dataset into csv format
        df_store.append(df)
    return df_store


class Visualize:
    def __init__(self):
        pass
    def adfuller_test(self, value):
        """
        Method Name: adfuller_test
        Description: This function carries out a statistical adfuller test and identifies if the data is
        stationary or non-stationary.
        Output: None
        """
        result=adfuller(value)
        labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
        for value,label in zip(result,labels):
            print(label+' : '+str(value) )
        if result[1] <= 0.05:
            print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data has no unit root and is stationary")
        else:
            print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")
    
    def data_visualize(self,data):
        """
        Method Name: data_visualize
        Description: This function crop the unnecessary intial part of the dataset.
        Output: df=clean dataset
                true_val_df='y' column values for sample where 'ds' column is null
        """
        true_val_df=[]
        df=[]
        for i,d in enumerate(data):
            true_val_df.append(d.loc[d['ds'].isnull()]['y'])
            # remove outlier data from the samples
            if i==0:
                d=d.loc[~(d['ds']=='1970-01-01 04:29:36')]
            elif i==1:
                d=d.loc[~(d['ds']=='1970-01-01 04:29:28.000')]
            d.dropna(inplace=True)
            df.append(d)
            #d=d.set_index('ds')
            d.reset_index(inplace=True)
            d.drop(['ds','index'],axis=1,inplace=True)
            # visualize the samples
            d.plot()
            plt.title('Sample {x}'.format(x=i+1))
            plt.xlabel('Time')
            plt.ylabel('Power in watt')
            #plt.savefig('sample_{x}_{y}.png'.format(x=i,y=message))
            plt.show()
            print('\nadfuller results for sample 1\n')
            self.adfuller_test(df[i]['y']) # check if data is stationary or not
        return df,true_val_df
    

def MAE(y,yhat):
    """
        Method Name: MAE
        Description: This function calculates the mean-squared error for the model by taking into consideration of 
        actual and the predicted values. 
        Output: df_store= mae=Mean_squared_error value
    """
    diff = np.abs(np.array(y)-np.array(yhat))
    try:
        mae =  round(np.mean(np.fabs(diff)),3)
    except:
        print("Error while calculating")
        mae = np.nan
    return mae

# Clean the sample data, visuallize it and check if stationary or non-stationary

In [ ]:
visual=Visualize()
df_stg=data_cleaning(file)
df=[]
df.append(df_stg[0][190:])
df.append(df_stg[1][190:])
df.append(df_stg[2][190:])
df,true_val_df_vs4=visual.data_visualize(df)

# Identify the p and q values for the samples for arima

In [ ]:
for i,v in enumerate(df):
    print('sample {}'.format(i+1))
    fig = plt.figure(figsize=(12,8))
    ax1 = fig.add_subplot(211)
    fig = sm.graphics.tsa.plot_acf(v['y'].copy(),ax=ax1,lags=250)  # plot the autocorrelation
    ax2 = fig.add_subplot(212)
    fig = sm.graphics.tsa.plot_pacf(v['y'].dropna(),ax=ax2) # plot the partial autocorrelation
    plt.show()


# Arima Model

In [ ]:
import statsmodels.api as sm
arima_score={'sample':[],'mean_score':[],'MAE_score':[],'p':[],'d':[],'q':[]}
sample_1_p=[1,17]
sample_2_p=[1,7,11]
sample_3_p=[2,8]
sample_1_q=[21]
sample_2_q=[21,18]
sample_3_q=[40,34,37]
arima_pram={'0':[sample_1_p,sample_1_q,0],'1':[sample_2_p,sample_2_q,0],'2':[sample_3_p,sample_3_q,0]}
for i,j in arima_pram.items():
    arima_train=pd.DataFrame(df[int(i)]['y'][:-20].copy())
    arima_test=pd.DataFrame(df[int(i)]['y'][-21:].copy())
    for m in j[0]:
        for n in j[1]:
            print('Sample{}'.format(int(i)+1)
            model=sm.tsa.ARIMA(arima_train['y'],order=(m,j[1],n))
            model=model_ARIMA.fit()
            print(model.summary())
            pred=model.predict(start=arima_test.index[0],end=arima_test.index[-1])
            mae_score=MAE(arima_test,pred)
            mean_score=arima_train['y'].mean()
            sample_name='Sample_{}'.format(int(i)+1)
            model_score['sample'].append(sample_name)
            model_score['mean_score'].append(mean_score)
            model_score['MAE_score'].append(mae_score)
            model_score['p'].append(m)
            model_score['d'].append(j[1])
            model_score['q'].append(n)
            plt.plot(arima_test.index,pred,label='Prediction')
            plt.plot(arima_test.index,arima_train['y'][-21:],label='Original')
            plt.title('Sample_1 mean value={y}, MAE value={z},p_value={a},d_value={b},q_value={c}'.format(y=mean_score,z=mae_score,a=m,b=0,c=n))
            plt.legend(loc="upper left")
            plt.xlabel('Time')
            plt.ylabel('Power / watt')
            #plt.savefig('sample_1_predicted_arima_p={a}_d={b}_q={c}.png'.format(a=i,b=d,c=j))
            plt.show()